# Making CMJ mask out of medullary mask


In [ ]:
import cv2 
import numpy as np
import pandas as pd
import os
import re
import tifffile
import matplotlib.pyplot as plt
%matplotlib inline

## Functions

In [ ]:
def make_one_binary(img):
    ## Ensure that binary mask values are either 0 or 1 
        ## I could speed this up by just doing img[img > 0], but I want to know if any erroneous values are in the mask for some reason.
    img = img * np.uint8(1)
        ## Converts boolean values to 0 and 1 
        
    uni_img = np.sort(np.unique(img))
    if not np.all(uni_img == np.array([0,1], dtype= "uint8")):
        if max(uni_img) == 255:
            img[img > 0] = 1 
        else:
            raise ValueError("Max value in array is neither 1 nor 255")
    return img


def mask_cmj(img_dir         : str, ## Really str_or_path 
             medulla_name    : str, 
             whole_lobe_name : str,
             cmj_name        : str,
             cmj_distance    : int,
             XY_resolution   : float) -> None:
    medullary_mask = tifffile.imread(os.path.join(img_dir, medulla_name)).astype("uint8")

    ## Ensure that binary mask values are either 0 or 1 
    medullary_mask = make_one_binary(medullary_mask)
    
    ## Make a CMJ mask 
    pixel_distance = np.ceil(cmj_distance/XY_resolution) ## Convert distance in micrometers to pixels given the size of the pixel (XY_resolution)
    kernel_size = int(2 * pixel_distance + 1)
        ## 2x gets twice the distance on either side using a square kernel 
        ## No need to calculate derivatives nor tangents. 
        ## +1 to make the kernel size odd and have a clear center
    kernel      = np.ones((kernel_size, kernel_size), np.uint8)
    dilate_mask = cv2.dilate(medullary_mask, kernel, iterations=1).astype("uint8") ## larger
    erode_mask  = cv2.erode(medullary_mask,  kernel, iterations=1).astype("uint8") ## smaller

    dilation_diff = dilate_mask - medullary_mask
    erosion_diff  = medullary_mask - erode_mask
    cmj_mask = dilation_diff + erosion_diff
    tifffile.imwrite(os.path.join(img_dir, cmj_name), cmj_mask)
    
    ## Remove CMJ from medulla and cortex
    medulla_sans_cmj = medullary_mask - erosion_diff
    tifffile.imwrite(os.path.join(img_dir, "medulla_mask_sans_cmj.tif"), medulla_sans_cmj) 

    whole_lobe_mask = tifffile.imread(os.path.join(img_dir, whole_lobe_name)).astype("uint8")
    whole_lobe_mask = make_one_binary(whole_lobe_mask)
        ## I never defined a cortical mask, only a whole lobe mask 
    cortex_sans_cmj  = whole_lobe_mask - dilate_mask 
    tifffile.imwrite(os.path.join(img_dir, "cortex_mask_sans_cmj.tif"), cortex_sans_cmj)

    return None


def make_tissue_region_mask(img_dir           : str, 
                            cortex_mask_name  : str,
                            medulla_mask_name : str,
                            cmj_mask_name     : str,
                            region_out_name   : str):
    ## Load images 
    cortex_mask  = tifffile.imread(os.path.join(img_dir, cortex_mask_name)).astype("uint8")
    medulla_mask = tifffile.imread(os.path.join(img_dir, medulla_mask_name)).astype("uint8")
    cmj_mask     = tifffile.imread(os.path.join(img_dir, cmj_mask_name)).astype("uint8")

    ## Assign each tissue region a different value (background : 0, cortex : 1, cmj : 2, medulla : 3) 
    region_mask = cortex_mask + (cmj_mask * np.uint8(2)) +  (medulla_mask * np.uint8(3))
    tifffile.imwrite(os.path.join(img_dir, region_out_name), region_mask)


def add_tissue_region(img_dir : str, img_name : str, region_name : str, out_name : str) -> None:
    img = tifffile.imread(os.path.join(img_dir, img_name))
    region_mask = tifffile.imread(os.path.join(img_dir, region_name))

    img = np.concatenate((img, region_mask[np.newaxis,...]))

    tifffile.imwrite(os.path.join(img_dir, out_name), img)
    return None


def add_tissue_to_markers_df(img_dir : str, markers_name : str, out_name : str) -> None:
    markers_df = pd.read_csv(os.path.join(img_dir , markers_name))
    tissue_df  = pd.DataFrame({"cycle" : [0], "marker_name" : ["tissue"], "row_num" : [markers_df.shape[0]]})
    markers_df = pd.concat([markers_df, tissue_df])
    markers_df.to_csv(os.path.join(img_dir, out_name), index= False)
                      
    return None 

## Run images

CMJ distance of 100 $\mu m$ on either side of the hand-drawn cortical-medullary boundary. 

Inspired by Lei et al., 2011 (https://doi.org/10.1084/jem.20102327)

In [ ]:
raw_dir= "/stor/scratch/Ehrlich/Users/John/histocytometry/raw_images/images_2023-08-10"
img_dirs= [os.path.join(raw_dir, img_dir) for img_dir in os.listdir(raw_dir) if re.search("_[A-D]$", img_dir)] 
img_dirs.sort()

In [ ]:
## In general, this code could be faster by making a monolith function that doesn't read in images multiple times. 
    ## I've traded speed for simplicity for now.
cmj_distance = 50
for img_dir in img_dirs:
    print(f"Starting {img_dir}")

    mask_cmj(img_dir          = img_dir,
             medulla_name     = "cleaned_medulla_mask.tif",
             whole_lobe_name  = "whole_lobe_mask.tif",
             cmj_name         = "cmj_mask.tif",
             cmj_distance     = cmj_distance,
             XY_resolution    = 0.479)
    
    make_tissue_region_mask(img_dir           = img_dir, 
                            cortex_mask_name  = "cortex_mask_sans_cmj.tif",
                            medulla_mask_name = "medulla_mask_sans_cmj.tif",
                            cmj_mask_name     = "cmj_mask.tif",
                            region_out_name   = f"thymus_regions_{cmj_distance}um.tif")
    
    add_tissue_region(img_dir     = img_dir,
                      img_name    = "reordered_image.ome.tif",
                      region_name = f"thymus_regions_{cmj_distance}um.tif",
                      out_name    = f"reordered_image_w_thymus_regions_{cmj_distance}um.ome.tif")
    
    add_tissue_to_markers_df(img_dir      = img_dir, 
                             markers_name = "markers.csv",
                             out_name     = f"markers_tissue_{cmj_distance}um.csv")

### Making boundary line on either side of CMJ for manuscript ROIs

In [ ]:
def draw_cmj_boundaries(img_dir, medulla_name, cmj_name, pixel_distance, out_name):
    ## Inner CMJ boundary
    medullary_mask = tifffile.imread(os.path.join(img_dir, medulla_name)).astype("uint8")
    medullary_mask = make_one_binary(medullary_mask)
    
    kernel_size = int(2 * pixel_distance + 1)
    kernel      = np.ones((kernel_size, kernel_size), np.uint8)

    dilate_mask = cv2.dilate(medullary_mask, kernel, iterations=1).astype("uint8") ## larger
    erode_mask  = cv2.erode(medullary_mask,  kernel, iterations=1).astype("uint8") ## smaller

    dilation_diff = dilate_mask - medullary_mask
    erosion_diff  = medullary_mask - erode_mask
    thin_inner_line_mask = dilation_diff + erosion_diff

    ## Outer CMJ boundary
    cmj_mask = tifffile.imread(os.path.join(img_dir, cmj_name)).astype("uint8")
    cmj_mask = make_one_binary(cmj_mask)
    cmj_medulla_mask = medullary_mask + cmj_mask

    dilate_mask = cv2.dilate(cmj_medulla_mask, kernel, iterations=1).astype("uint8") ## larger
    erode_mask  = cv2.erode(cmj_medulla_mask,  kernel, iterations=1).astype("uint8") ## smaller

    dilation_diff = dilate_mask - cmj_medulla_mask
    erosion_diff  = cmj_medulla_mask - erode_mask
    thin_outer_line_mask = dilation_diff + erosion_diff

    ## Combined output
    thin_line_mask = thin_inner_line_mask + thin_outer_line_mask
    tifffile.imwrite(os.path.join(img_dir, out_name), thin_line_mask)

In [ ]:
pixel_distance= 8
    # I need to run 2 as well 
for img_dir in img_dirs:
    draw_cmj_boundaries(img_dir = img_dir, 
                        medulla_name = "medulla_mask_sans_cmj.tif", 
                        cmj_name     = "cmj_mask.tif", 
                        pixel_distance = pixel_distance, 
                        out_name       = f"cmj_boundaries_{pixel_distance}pixels.tif")
    print(f"Done w/ {img_dir}")